<a href="https://colab.research.google.com/github/dkbs12/External_test/blob/main/Phase02_LFQA_test_01_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,elasticsearch,inference,ocr,preprocessing,file-conversion,pdf]
pip install datasets>=2.6.1

apt install libgraphviz-dev
pip install pygraphviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Obtaining dependency information for farm-haystack[colab,elasticsearch,file-conversion,inference,ocr,pdf,preprocessing] from https://files.pythonhosted.org/packages/e1/21/5b8e501524275bbb4345cd66499310e035ff953e4192af1aaa007edf5b46/farm_haystack-1.21.2-py3-none-any.whl.metadata
  Obtaining dependency information for events from https://files.pythonhosted.org/packages/25/ed/e47dec0626edd468c84c04d97769e7ab4ea6457b7f54dcb3f72b17fcd876/Events-0.5-py3-none-any.whl.metadata
  Obtaining dependency information for httpx from https://files.pythonhosted.org/packages/33/0d/d9ce469af019741c8999711d36b270ff992ceb1a0293f73f9f34fdf131e9/httpx-0.25.0-py3-none-any.whl.metadata
  Obtaining dependency information for posthog from https://files.pythonhosted.org/packages/a7/73/35758818

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.8.0 which is incompatible.




In [ ]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [ ]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
# Connect to Elasticsearch
import os
import time

from haystack.document_stores.elasticsearch import ElasticsearchDocumentStore

# Wait 30 seconds only to be sure Elasticsearch is ready before continuing
time.sleep(30)

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")

document_store = ElasticsearchDocumentStore(host=host, username="", password="", index="document")

In [ ]:
from haystack.utils import fetch_archive_from_http, convert_files_to_docs
from haystack.nodes import PreProcessor

doc_dir = "data/Phase1_test_data_02"
url = "https://github.com/dkbs12/External_test/raw/main/Phase1_test_data_02.zip"
fetch_archive_from_http(url=url, output_dir=doc_dir)

# convert files to dicts containing documents that can be indexed to our datastore
got_docs = convert_files_to_docs(dir_path=doc_dir)

In [ ]:
preprocessor = PreProcessor(
    clean_whitespace=True,
    clean_header_footer=True,
    clean_empty_lines=True,
    split_by="word",
    split_length=200,
    split_overlap=20,
    split_respect_sentence_boundary=True,
)

all_docs = preprocessor.process(got_docs)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Preprocessing: 100%|██████████| 3/3 [00:00<00:00, 17.82docs/s]


In [ ]:
document_store.delete_documents()
document_store.write_documents(all_docs)

In [ ]:
from haystack.nodes import BM25Retriever, EmbeddingRetriever
from haystack.utils import print_answers

bm25_retriever = BM25Retriever(document_store=document_store)

embedding_retriever = EmbeddingRetriever(
    document_store=document_store, embedding_model="sentence-transformers/multi-qa-mpnet-base-dot-v1"
)

document_store.update_embeddings(embedding_retriever, update_existing_embeddings=False)

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Updating embeddings:   0%|          | 0/103 [00:00<?, ? Docs/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Updating embeddings: 10000 Docs [00:14, 710.16 Docs/s]


In [ ]:
from haystack.nodes import PromptNode, PromptTemplate, AnswerParser

prompt_template = PromptTemplate(
    prompt="""
    Answer the question truthfully based solely on the given documents.
    If the documents do not contain the answer to the question, say that answering is not possible given the available information.
    Your answer should be no longer than 50 words.
    Documents:{join(documents)}
    Question:{query}
    Answer:
    """,
    output_parser=AnswerParser(),
)

prompt_node = PromptNode(
    model_name_or_path="text-davinci-003", api_key=api_key, default_prompt_template=prompt_template
)

In [ ]:
from haystack.pipelines import Pipeline
from haystack.nodes import JoinDocuments

# Create ensembled pipeline
p_ensemble = Pipeline()
p_ensemble.add_node(component=bm25_retriever, name="BM25Retriever", inputs=["Query"])
p_ensemble.add_node(component=embedding_retriever, name="EmbeddingRetriever", inputs=["Query"])
p_ensemble.add_node(
    component=JoinDocuments(join_mode="concatenate"), name="JoinResults", inputs=["BM25Retriever", "EmbeddingRetriever"]
)
p_ensemble.add_node(component=prompt_node, name="prompt_node", inputs=["JoinResults"])

# Uncomment the following to generate the pipeline image
# p_ensemble.draw("pipeline_ensemble.png")

In [ ]:
# Run pipeline
res = p_ensemble.run(
    query="What is NDC?", params={"EmbeddingRetriever": {"top_k": 5}, "BM25Retriever": {"top_k": 5}}
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
print_answers(res, details="medium")

'Query: What is NDC?'
'Answers:'
[   {   'answer': 'NDC is an XML-based communication standard created by the '
                  'International Air Transportation Association (IATA) to let '
                  'airlines bring rich content and ancillaries directly to '
                  'online travel agencies, TMCs, and other flight resellers '
                  'through a set of travel APIs. It enables personalized offer '
                  'creation and flexible, real-time ticket pricing.'}]


In [ ]:
res['documents']

[<Document: {'content': 'Low-performance, slow, and glitchy software with a non-intuitive user interface may damage the NDC’s reputation making vendors come back to old distribution schemes.\n\nPoor after-sales service. “An Achilles’ heel of NDC” — that’s how Ksenia Penkova refers to after-sales service (order cancellations and changes, making refunds, etc.): “Those processes must be automated wherever possible to minimize the number of contacts with the support team.” While some carriers already provide after-sales functionality via NDC APIs, others have only started this journey; so in many cases servicing passengers after a purchase is completed poses a problem.\n\nWith all these details in mind, it’s time to decide on the NDC implementation strategy that will fit your business best.\n\nWhat to consider when choosing NDC content providers\nHere are some insights from industry experts to help you choose the best approach to NDC implementation and define the right partners.\n\nDon’t w

In [ ]:
meta_args = [x.to_dict() for x in res["documents"]]
[x['meta']['name'] for x in meta_args]

['New_Distribution_Capability_in_Air_Travel.txt',
 'Transitioning_to_a_Future_of_Intelligent_Dynamic_Offers.docx',
 'New_Distribution_Capability_in_Air_Travel.txt',
 'Dynamic_pricing_of_airline_offers.pdf',
 'New_Distribution_Capability_in_Air_Travel.txt',
 'New_Distribution_Capability_in_Air_Travel.txt',
 'New_Distribution_Capability_in_Air_Travel.txt']